# Gallery Example: Erl/Erl/1 Queue

This example demonstrates an Erlang/Erlang/1 queueing system:
- **Arrivals**: Erlang distribution
- **Service**: Erlang distribution
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

Both arrival and service processes use Erlang distributions, providing lower variability than exponential. This model also demonstrates class switching.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_erlerl1(n=5): Network {
    """Create Erl/Erl/1 queueing model with class switching"""
    val model = Network("Erl/Erl/1")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
    val oclass2 = OpenClass(model, "Class2")
// Erlang arrival for Class1, Class2 is disabled at source
    source.setArrival(oclass1, Erlang.fit_mean_and_order(1, n))
    source.setArrival(oclass2, Disabled())
// Erlang service for Class1, Exponential for Class2
    queue.setService(oclass1, Erlang.fit_mean_and_order(0.5, n))
    queue.setService(oclass2, Exp(3))
// Block 3: topology with class switching
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addRoute(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model
// Create the model
val model = gallery_erlerl1()

## About Erlang Distributions

The Erlang distribution is a sum of n independent exponential random variables:
- **Mean**: k/λ where k is the order
- **Variance**: k/λ²
- **SCV (squared coefficient of variation)**: 1/k

With order n=5, the SCV is 0.2, providing much less variability than exponential (SCV=1).

## About Class Switching

This model demonstrates class switching:
- Jobs arrive as Class1 (Erlang arrivals)
- After service, jobs become Class2 and leave
- Class2 cannot arrive from the source (Disabled)

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)